In [ ]:
import pydantic


2.10.6


In [4]:
class User:
    def __init__(self,id: int,name="akif agayev"):
        if not isinstance(id,int):
            raise TypeError(f"Expected id tobe an int {type(id).__name__}")
        
        if not isinstance(name,str):
            raise TypeError(f"Expected name tobe an str {type(name).__name__}")
        

        self.id = id
        self.name = name

try:
    user = User("1254")   
except TypeError as e:
    print(e)





Expected id tobe an int str


In [12]:
from pydantic import BaseModel


class User(BaseModel):

    id : int
    name : str = "malik hesenov"


user = User(id="45") 
print(user)   
print(user.model_fields_set)
user = User(id=45,name="hafiz haciyev")
print(user.model_fields_set)

id=45 name='malik hesenov'
{'id'}
{'name', 'id'}


In [16]:
print(user.model_dump())
print(user.model_dump_json())
print(user.model_json_schema())

{'id': 45, 'name': 'hafiz haciyev'}
{"id":45,"name":"hafiz haciyev"}
{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'default': 'malik hesenov', 'title': 'Name', 'type': 'string'}}, 'required': ['id'], 'title': 'User', 'type': 'object'}


In [ ]:
from typing import Optional,List


class Food(BaseModel):
    name : str
    price : float
    ingridients : Optional[List[str]] = None


class Resturant(BaseModel):
    name: str
    location: str
    foods: List[Food] 


resturant_instance = Resturant(
    name="abseron mariot",
    location="Baki",
    foods=[
        {'name':"Cheese pizza","price":45,"ingridients":["Cheese","tomato","Dough"]},
        {'name':"Rice","price":852,"ingridients":["rice","poteto","Dough"]},
        {'name':"Rice-souse","price":552}
    ]
)       


print(resturant_instance)
print(resturant_instance.model_dump())


name='abseron mariot' location='Baki' foods=[Food(name='Cheese pizza', price=45.0, ingridients=['Cheese', 'tomato', 'Dough']), Food(name='Rice', price=852.0, ingridients=['rice', 'poteto', 'Dough']), Food(name='Rice-souse', price=552.0, ingridients=None)]
{'name': 'abseron mariot', 'location': 'Baki', 'foods': [{'name': 'Cheese pizza', 'price': 45.0, 'ingridients': ['Cheese', 'tomato', 'Dough']}, {'name': 'Rice', 'price': 852.0, 'ingridients': ['rice', 'poteto', 'Dough']}, {'name': 'Rice-souse', 'price': 552.0, 'ingridients': None}]}


In [ ]:
!pip install pydantic[email]


In [30]:
from typing import List
from pydantic import BaseModel,EmailStr,PositiveInt,conlist,Field,HttpUrl
import re

class Address(BaseModel):
    street : str
    city : str
    state : str
    zip_code : str


class Employee(BaseModel):
    name : str
    position: str
    email : EmailStr

class Owner(BaseModel):
    name: str
    email: EmailStr



class Resturant(BaseModel):
    name :str = Field(...,pattern=r"^[a-zA-Z0-9]+$")
    owner : Owner
    address : Address
    employees : conlist(Employee ,min_length=2)
    number_of_seats : PositiveInt
    delivery : bool
    website : HttpUrl


resturant_instance_1 = Resturant(
    name="Tastyparty",
    owner={
        "name":"eldar qasimov",
        "email":"eldar@mail.ru"
    },
    address={
        "street":"h.cavid",
        "city":"baku",
        "state":"class",
        "zip_code":"456"
    },
    employees=[
        {"name":"david collins",
        "position":"servant",
        "email":"collins@gmail.com"},

        {"name":"david bakhem",
        "position":"CEO",
        "email":"beckhams@gmail.com"}
        ],
    number_of_seats= 45,
    delivery=True,
    website="https://www.website.com"
        )    


In [ ]:
from pydantic import BaseModel,EmailStr,field_validator

class Owner(BaseModel):
    name: str
    email: EmailStr

    @field_validator("name")
    @classmethod
    def name_must_contain_space(cls,v:str) -> str:

        if " " not in v:
            raise ValueError("Owner name must contain a space")
        return v.title
    

try:
    owner_instance = Owner(name="eldar",email="eldar@gmail.com") 
except ValueError as e:
    print(e)  
